# LoRA Hyperparameter Optimization for Knowledge Update

## Минимизация катастрофического забывания при дообучении языковых моделей

Этот notebook демонстрирует полный пайплайн оптимизации гиперпараметров LoRA для задачи обновления фактологических знаний с минимизацией катастрофического забывания.

**Автор:** Спирин К.Г.  
**Организация:** AIRI, Школа Летово  
**Дата:** 2025

## 1. Установка зависимостей

Установим необходимые библиотеки для работы.

In [ ]:
# Установим только отсутствующие пакеты (быстрее при повторных запусках)
import importlib, subprocess, sys

required = [
    'unsloth', 'optuna', 'datasets', 'transformers', 'trl', 'torch', 'accelerate',
    'matplotlib', 'seaborn', 'pandas', 'numpy', 'tqdm'
]

def install(pkg):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', pkg])

for pkg in required:
    try:
        importlib.import_module(pkg)
    except Exception:
        print(f'Installing {pkg}...')
        install(pkg)

print('Установка зависимостей завершена (или пакеты уже были установлены).')


## 2. Импорты и настройка

In [ ]:
import torch
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, load_from_disk
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Опциональные инструменты
from tqdm import tqdm

# Импорт наших модулей (убедитесь, что файлы присутствуют в рабочей директории)
try:
    from generate_dataset import DatasetGenerator, GenerationConfig, KnowledgeClassifier
    from train_lora import LoRATrainer
    from optimization_pipeline import KnowledgeShiftCalculator, run_optimization
    from utils import (
        plot_optimization_history,
        plot_parameter_importance,
        plot_param_relationships,
        plot_knowledge_shifts,
        create_results_summary,
        export_best_config
    )
except Exception as e:
    print('Некоторые утилиты не найдены. Убедитесь, что файлы generate_dataset.py, train_lora.py, optimization_pipeline.py и utils.py в той же папке.')
    print('Ошибка импорта:', e)

# Настройка для воспроизводимости
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Проверка доступности GPU и базовые оптимизации
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    try:
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    except Exception:
        pass
    # Быстрые оптимизации
    torch.backends.cudnn.benchmark = True
    # Не делаем deteministic=True чтобы не терять производительность

print('\nИмпорт и первичная настройка завершены.')


## 3. Загрузка и подготовка данных

### 3.1 Загрузка базового датасета

Используем предварительно подготовленный датасет с категоризацией знаний.

In [ ]:
# Загрузка датасета
# Если датасет уже есть локально:
dataset_path = "./my_dataset"
if os.path.exists(dataset_path):
    try:
        dataset = load_from_disk(dataset_path)
        print(f"Dataset loaded from {dataset_path}")
    except Exception as e:
        print('Не удалось загрузить датасет из disk:', e)
        dataset = None
else:
    print('Локальный путь не найден, попробуйте загрузить с HuggingFace или сохраните датасет в ./my_dataset')
    dataset = None

# Или загрузка с HuggingFace (раскомментируйте при необходимости):
# dataset = load_dataset("s-nlp/Llama-3.1-8B-Instruct-DBpedia-HighlyKnown")
# dataset.save_to_disk(dataset_path)

if dataset is not None and 'full' in dataset:
    print(f"Dataset loaded with {len(dataset['full'])} examples")
else:
    print('Dataset отсутствует или имеет неожиданную структуру. Убедитесь, что dataset["full"] существует.')


### 3.2 Анализ распределения категорий знаний

In [ ]:
# Подсчет категорий
if dataset is not None and 'full' in dataset:
    categories = [item.get('Category', 'Unknown') for item in dataset['full']]
    category_counts = pd.Series(categories).value_counts()

    print("Распределение категорий знаний:")
    print(category_counts)
    print(f"\nВсего примеров: {len(categories)}")
else:
    print('Пропускаем анализ распределения — датасет не загружен.')


In [ ]:
# Визуализация распределения (если есть данные)
if 'category_counts' in globals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Bar plot
    category_counts.plot(kind='bar', ax=ax1)
    ax1.set_title('Распределение категорий знаний', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Категория', fontsize=12)
    ax1.set_ylabel('Количество', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')

    # Pie chart
    category_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Процентное соотношение', fontsize=14, fontweight='bold')
    ax2.set_ylabel('')

    plt.tight_layout()
    plt.savefig('category_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print('Нет данных для визуализации.')


### 3.3 Примеры из каждой категории

In [ ]:
# Получаем примеры из каждой категории
if dataset is not None and 'full' in dataset:
    examples_by_category = defaultdict(list)
    for item in dataset['full']:
        cat = item.get('Category', 'Unknown')
        if len(examples_by_category[cat]) < 3:
            examples_by_category[cat].append(item)

    # Выводим примеры
    for category in ['HighlyKnown', 'MaybeKnown', 'WeaklyKnown', 'Unknown']:
        print(f"\n{'='*80}")
        print(f"Категория: {category}")
        print('='*80)
        
        for i, example in enumerate(examples_by_category.get(category, []), 1):
            print(f"\nПример {i}:")
            print(f"  Вопрос: {example.get('question', '')}")
            
            # Извлекаем ответ
            answer = example.get('answer', '')
            if isinstance(answer, dict):
                answer_text = answer.get('normalized_aliases', [''])[0] if answer.get('normalized_aliases') else str(answer)
            elif isinstance(answer, list) and len(answer) > 0:
                answer_text = answer[0].get('normalized_aliases', [''])[0] if isinstance(answer[0], dict) else str(answer[0])
            else:
                answer_text = str(answer)
            
            print(f"  Ответ: {answer_text}")
else:
    print('Пропускаем — датасет не загружен.')


## 4. Генерация baseline датасета

Создадим baseline датасет для последующего сравнения после обучения.

In [ ]:
# ВНИМАНИЕ: Этот шаг требует значительного времени (~4-6 часов)
# Рекомендуется использовать уже сгенерированный датасет

GENERATE_BASELINE = False  # Установите True для генерации

if GENERATE_BASELINE:
    print("Генерация baseline датасета...")
    print("ВНИМАНИЕ: Это займет несколько часов!")
    
    config = GenerationConfig(
        n_shot=4,
        n_experiments=10,
        batch_size_greedy=256,
        batch_size_sample=16
    )
    
    generator = DatasetGenerator(
        model_name="unsloth/Qwen3-0.6B-Base",
        config=config
    )
    
    baseline_dataset = generator.generate_dataset(
        data_path="s-nlp/Llama-3.1-8B-Instruct-DBpedia-HighlyKnown",
        output_path="./baseline_dataset",
        use_sampling=False,
        max_examples=500
    )
    
    print("Baseline датасет сгенерирован!")
else:
    print("Используем существующий baseline датасет")
    print("Убедитесь, что датасет находится в ./baseline_dataset")


## 5. Пример обучения одной конфигурации

Прежде чем запускать полную оптимизацию, попробуем обучить одну модель с фиксированными параметрами.

In [ ]:
# Создаем аргументы для обучения
class Args:
    def __init__(self):
        self.data_path = "./my_dataset"
        self.unknown = 500
        self.high_known = 1
        self.rank = 1
        self.learning_rate = 1e-5
        self.lora_alpha = 0.1
        self.dropout = 0.1
        self.seed = 42
        self.paraphrase = False

args = Args()

print("Конфигурация обучения:")
print(f"  Rank: {args.rank}")
print(f"  Learning Rate: {args.learning_rate}")
print(f"  Alpha: {args.lora_alpha}")
print(f"  Unknown facts: {args.unknown}")
print(f"  HighlyKnown facts: {args.high_known}")


In [ ]:
# ВНИМАНИЕ: Обучение занимает ~2-3 часа на T4 GPU
TRAIN_EXAMPLE = False  # Установите True для обучения

if TRAIN_EXAMPLE:
    print("Начинаем обучение...")
    
    trainer = LoRATrainer(args)
    stats, model_path = trainer.train()
    
    print(f"\nОбучение завершено!")
    print(f"Модель сохранена в: {model_path}")
else:
    print("Обучение пропущено. Установите TRAIN_EXAMPLE=True для запуска.")


## 6. Вычисление Knowledge Shifts

Демонстрация работы калькулятора knowledge shifts.

In [ ]:
# Создаем калькулятор
try:
    shift_calculator = KnowledgeShiftCalculator()
    
    print("Веса для различных типов сдвигов:")
    print("\nПозитивные сдвиги (желательные):")
    print(f"  UK → HK: +{shift_calculator.shift_types['UK_to_HK']} (полное усвоение нового факта)")
    print(f"  UK → MK: +{shift_calculator.shift_types['UK_to_MK']} (частичное усвоение)")
    print(f"  MK → HK: +{shift_calculator.shift_types['MK_to_HK']} (улучшение частично известного)")
    
    print("\nНегативные сдвиги (нежелательные):")
    print(f"  HK → UK: {shift_calculator.shift_types['HK_to_UK']} (катастрофическое забывание)")
    print(f"  HK → MK: {shift_calculator.shift_types['HK_to_MK']} (частичная деградация)")
    print(f"  MK → UK: {shift_calculator.shift_types['MK_to_UK']} (потеря частичных знаний)")
except Exception as e:
    print('Не удалось создать KnowledgeShiftCalculator — убедитесь, что optimization_pipeline.py присутствует и содержит соответствующий класс.')
    print('Ошибка:', e)


In [ ]:
# Пример вычисления shifts (если есть обученная модель)
CALCULATE_SHIFTS = False  # Установите True если есть данные

if CALCULATE_SHIFTS:
    # Пути к датасетам
    baseline_path = "./baseline_dataset"
    after_training_path = "./after_training_dataset"
    
    # Вычисляем сдвиги
    shifts = shift_calculator.calculate_shifts_from_datasets(
        baseline_path, 
        after_training_path
    )
    
    # Вычисляем score
    score = shift_calculator.calculate_objective_score(shifts)
    
    print("\nРезультаты Knowledge Shifts:")
    print("="*50)
    for shift_type, count in shifts.items():
        weight = shift_calculator.shift_types.get(shift_type, 0)
        print(f"{shift_type}: {count} (вес: {weight})")
    
    print(f"\nИтоговый weighted score: {score:.3f}")
    
    # Визуализация
    plot_knowledge_shifts(shifts, save_path='example_shifts.png')
else:
    print("Вычисление shifts пропущено. Требуются baseline и post-training датасеты.")


## 7. Запуск оптимизации гиперпараметров

### 7.1 Настройка оптимизации

**ВНИМАНИЕ:** Полная оптимизация с 28 trials требует ~70-80 часов GPU времени!

In [ ]:
# Параметры оптимизации
BASE_DATASET = "./baseline_dataset"
TEST_QUESTIONS = "s-nlp/Llama-3.1-8B-Instruct-DBpedia-HighlyKnown"
OUTPUT_DIR = "./optimization_results"
N_TRIALS = 28  # Уменьшите для быстрого тестирования
STUDY_NAME = "lora_optimization"

print(f"Параметры оптимизации:")
print(f"  Количество trials: {N_TRIALS}")
print(f"  База данных: {OUTPUT_DIR}/optuna.db")
print(f"  Примерное время: ~{N_TRIALS * 2.5:.1f} часов")


In [ ]:
# ЗАПУСК ОПТИМИЗАЦИИ
RUN_OPTIMIZATION = False  # Установите True для запуска

if RUN_OPTIMIZATION:
    print("Запуск оптимизации...")
    print("Это займет значительное время. Прогресс будет логироваться.")
    
    study = run_optimization(
        base_dataset_path=BASE_DATASET,
        test_questions_path=TEST_QUESTIONS,
        output_dir=OUTPUT_DIR,
        n_trials=N_TRIALS,
        study_name=STUDY_NAME
    )
    
    print("\nОптимизация завершена!")
else:
    print("Оптимизация пропущена.")
    print("Для запуска установите RUN_OPTIMIZATION=True")
    print("\nДля загрузки существующего study используйте:")
    print("study = optuna.load_study(study_name=STUDY_NAME, storage=...)")


### 7.2 Загрузка результатов оптимизации

Если оптимизация уже проведена, загрузим результаты.

In [ ]:
# Загрузка существующего study
try:
    storage = f"sqlite:///{OUTPUT_DIR}/optuna.db"
    study = optuna.load_study(
        study_name=STUDY_NAME,
        storage=storage
    )
    
    print(f"Study загружен успешно!")
    print(f"Количество trials: {len(study.trials)}")
    print(f"Лучший score: {study.best_value:.4f}")
    
except Exception as e:
    print(f"Не удалось загрузить study: {e}")
    print("Запустите оптимизацию или проверьте путь к базе данных")
    study = None


## 8. Анализ результатов оптимизации

### 8.1 Общая статистика

In [ ]:
if study is not None:
    # Лучший trial
    best_trial = study.best_trial
    
    print("="*80)
    print("ЛУЧШАЯ КОНФИГУРАЦИЯ")
    print("="*80)
    print(f"\nTrial номер: {best_trial.number}")
    print(f"Score: {best_trial.value:.4f}")
    print("\nГиперпараметры:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")
    
    if best_trial.user_attrs:
        print("\nДополнительные метрики:")
        for key, value in best_trial.user_attrs.items():
            print(f"  {key}: {value}")
    
    # Экспорт конфигурации
    try:
        export_best_config(study, f"{OUTPUT_DIR}/best_config.json")
    except Exception:
        print('Не удалось экспортировать конфигурацию — проверьте, доступна ли функция export_best_config')
else:
    print("Study не загружен. Пропускаем анализ.")


### 8.2 История оптимизации

In [ ]:
if study is not None:
    plot_optimization_history(study, save_path=f"{OUTPUT_DIR}/optimization_history.png")
else:
    print("Study не загружен")


### 8.3 Важность параметров

In [ ]:
if study is not None:
    plot_parameter_importance(study, save_path=f"{OUTPUT_DIR}/parameter_importance.png")
else:
    print("Study не загружен")


### 8.4 Влияние отдельных параметров

In [ ]:
if study is not None:
    # Анализ влияния rank
    plot_param_relationships(study, 'rank', save_path=f"{OUTPUT_DIR}/rank_effect.png")
    
    # Анализ влияния learning_rate
    plot_param_relationships(study, 'learning_rate', save_path=f"{OUTPUT_DIR}/lr_effect.png")
    
    # Анализ влияния alpha
    plot_param_relationships(study, 'lora_alpha', save_path=f"{OUTPUT_DIR}/alpha_effect.png")
else:
    print("Study не загружен")


### 8.5 Таблица всех результатов

In [ ]:
if study is not None:
    # Создаем сводную таблицу
    results_df = create_results_summary(study)
    
    # Показываем топ-10 trials
    print("\nТОП-10 КОНФИГУРАЦИЙ:")
    print("="*100)
    
    display_cols = ['number', 'value', 'params_rank', 'params_learning_rate', 'params_lora_alpha']
    if 'user_attrs_positive_shifts' in results_df.columns:
        display_cols.extend(['user_attrs_positive_shifts', 'user_attrs_negative_shifts'])
    
    available_cols = [col for col in display_cols if col in results_df.columns]
    print(results_df.head(10)[available_cols].to_string(index=False))
    
    # Сохраняем полную таблицу
    results_df.to_csv(f"{OUTPUT_DIR}/all_results.csv", index=False)
    print(f"\nПолная таблица сохранена в {OUTPUT_DIR}/all_results.csv")
else:
    print("Study не загружен")


## 9. Практические рекомендации

На основе проведенных экспериментов можно сформулировать следующие рекомендации.

In [ ]:
if study is not None:
    try:
        best_params = study.best_params
    except Exception:
        best_params = None
    
    print("="*80)
    print("ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ")
    print("="*80)
    
    if best_params is not None:
        print("\n1. ОПТИМАЛЬНЫЕ ГИПЕРПАРАМЕТРЫ для обновления фактологических знаний:")
        print(f"   - Rank: {best_params.get('rank')}")
        print(f"   - Learning Rate: {best_params.get('learning_rate')}")
        print(f"   - Alpha: {best_params.get('lora_alpha')}")
    else:
        print('Нет доступных лучших параметров — загрузите study.')
    
    print("\n2. ОБЩИЕ ПРИНЦИПЫ:")
    print("   - Используйте минимально возможный rank и small alpha для уменьшения вмешательства в базовую модель.")
    print("   - Применяйте небольшой learning rate (1e-5 .. 5e-5) и короткие эпохи при LoRA дообучении для снижения забывания.")
    print("   - Тестируйте на baseline датасете, чтобы контролировать деградацию производительности.")
else:
    print("Study не загружен — рекомендации ограничены.")


## Быстрые улучшения и переключатели для тестирования

Эти флаги позволяют быстро запустить упрощённую версию pipeline для проверки, не тратя часы GPU.

In [ ]:
# Быстрые переключатели
QUICK_DEBUG = True   # Уменьшает N_TRIALS и примеры для быстрого прогона
USE_AMP = True       # Использовать смешанную точность (если доступно)

if QUICK_DEBUG:
    print('QUICK_DEBUG включён — уменьшаем нагрузку.')
    N_TRIALS = 4
    # Можно ограничить количество примеров при генерации/оценке
    MAX_EXAMPLES_QUICK = 50

if USE_AMP and torch.cuda.is_available():
    print('Mixed precision enabled (используйте torch.cuda.amp в обучении)')
else:
    print('Mixed precision недоступна или отключена.')
